### 랭체인 도구로 에이전트 만들기

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4o")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈어요. 감사합니다! 어떻게 지내셨나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 12, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4kvPLL3EK1Bb5ksICg6m75HeIwDm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9fda1f4c-b15f-4792-9241-856a90fa6389-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [5]:
# .bind_tools() 메서드를 사용하여 도구를 LLM에 바인딩

# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [6]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage(
        content="당신은 현재 시각을 알려주는 도구를 사용할 수 있는 AI 어시스턴트입니다. "
                "사용자가 요청한 타임존과 지역명에 따라 현재 시각을 반환하세요."
    ),
    HumanMessage(
        content="서울의 현재 시각을 알려줘."
    )
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='당신은 현재 시각을 알려주는 도구를 사용할 수 있는 AI 어시스턴트입니다. 사용자가 요청한 타임존과 지역명에 따라 현재 시각을 반환하세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='서울의 현재 시각을 알려줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZVvYLJhhk98pmVYJyyjkttJN', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"서울"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 161, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_ea40d5097a', 'id': 'chatcmpl-C4ky12LYGFcDe3WzuNR65qnBIVSAW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--06806981-f017

In [16]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("서울과 뉴욕은 지금 몇시야?"),
]

response = llm_with_tools.invoke(messages)
messages.append(response)

for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(selected_tool.name, ' >>> ', tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)
    
print(messages)

get_current_time  >>>  {'timezone': 'Asia/Seoul', 'location': '서울'}
Asia/Seoul (서울) 현재시각 2025-08-15 18:33:20 
get_current_time  >>>  {'timezone': 'America/New_York', 'location': '뉴욕'}
America/New_York (뉴욕) 현재시각 2025-08-15 05:33:20 
[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='서울과 뉴욕은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IHjqXfn0ScgEWWQb49mthzQF', 'function': {'arguments': '{"timezone": "Asia/Seoul", "location": "서울"}', 'name': 'get_current_time'}, 'type': 'function'}, {'id': 'call_mhsHmRrP899KmbEfNvmQjnDR', 'function': {'arguments': '{"timezone": "America/New_York", "location": "뉴욕"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 137, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0

In [17]:
llm_with_tools.invoke(messages)  # (10) 도구 호출 결과를 포함한 메시지를 LLM에 전달하여 최종 응답 생성

AIMessage(content='지금 서울은 2025년 8월 15일 18시 33분이고, 뉴욕은 같은 날 5시 33분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 285, 'total_tokens': 321, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4l6JPXBajgcn3etoJjHdgSC18Upv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--867d37b1-1811-49af-b3cd-df4112d2a94f-0', usage_metadata={'input_tokens': 285, 'output_tokens': 36, 'total_tokens': 321, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 파이단틱(pydantic) 이용하기

pydantic은 입력된 데이터의 유효성과 형식을 검증하고 특정 데이터 형식으로 명확하게 표현할 때 사용하는 라이브러리입니다.

07 장의 get_yf_stock_history 함수에 사용되는 ticker 와 period 를 매개변수로 받는데,  
이를 pydantic의 BaseModel 과 Field 를 이용해 더욱 명확하게 정의할 수 있다.

In [18]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [24]:
import yfinance as yf

# def get_yf_stock_history(ticker: str, period: str):
#     stock = yf.Ticker(ticker)
#     history = stock.history(period=period)
#     history_md = history.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
#     print(history_md)
#     return history_md

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput):
    """ 주식 종목의 가격 데이터를 조회하는 함수 """
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
    # print(history_md)
    return history_md

llm = ChatOpenAI(model="gpt-4o")

# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time, get_yf_stock_history,]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.")
]

# messages.append(HumanMessage("서울과 뉴욕은 지금 몇시야?"))
# response = llm_with_tools.invoke(messages)
# print(response)

# messages.append(response)

messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가가 올랐나? 내렸나?"))
response = llm_with_tools.invoke(messages)
print(response)

messages.append(response)

for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # (7) tool_dict를 사용하여 도구 함수를 선택
    print(selected_tool.name, ' >>> ', tool_call["args"]) # (8) 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # (9) 도구 함수를 호출하여 결과를 반환
    print(tool_msg)  # 도구 호출 결과 출력 
    messages.append(tool_msg)


content='' additional_kwargs={'tool_calls': [{'id': 'call_rXVV61A8qOtJdL6pkrEdFtTH', 'function': {'arguments': '{"stock_history_input":{"ticker":"TSLA","period":"1mo"}}', 'name': 'get_yf_stock_history'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 215, 'total_tokens': 242, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4n4U2ZZeB57slTdTbvV9cDBBEskR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--005c8c80-ed9f-421e-af8a-3cbc960e1a46-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_rXVV61A8qOtJdL6pkrEdFtTH', 'type': 'tool_call'}] usage_metadata={'input_to

In [25]:
llm_with_tools.invoke(messages)  # (10) 도구 호출 결과를 포함한 메시지를 LLM에 전달하여 최종 응답 생성

AIMessage(content='한 달 전 테슬라의 종가가 $310.78이었고, 어제의 종가는 $335.58입니다. 따라서 지난 한 달 동안 테슬라의 주가는 상승하였습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 1632, 'total_tokens': 1677, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4n4pbUfxovEHixKYXiCshvj47J4m', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c121c439-c26d-441e-a899-121906c61017-0', usage_metadata={'input_tokens': 1632, 'output_tokens': 45, 'total_tokens': 1677, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})